In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

In [2]:
underground_activity = pd.read_csv("data/london_underground_activity.csv")
underground_stations = pd.read_csv("data/london_underground_station_info.csv")

In [4]:
underground_activity.head()

,year,nlc,station,area,weekday_entry,weekday_exit,saturday_entry,saturday_exit,sunday_entry,sunday_exit,annual_entry_and_exit
0,2007,500,Acton Town,Ealing,9205,8899.0,6722,6320,4427,4304,5.77
1,2007,502,Aldgate,City of London,9887,10397.0,2191,2587,1484,1772,5.57
2,2007,503,Aldgate East,Tower Hamlets,12820,12271.0,7040,6220,5505,5000,7.66
3,2007,505,Alperton,Brent,4611,4719.0,3354,3450,2433,2503,3.01
4,2007,506,Amersham,Chiltern,4182,3938.0,1709,1585,1004,957,2.34


In [5]:
underground_stations.head()

,station,line,local_authority,zone,opened
0,Acton Town,District,Ealing,3,1879-07-01
1,Aldgate,Metropolitan[a],City of London,1,1876-11-18
2,Aldgate East,Hammersmith & City[d],Tower Hamlets,1,1884-10-06
3,Alperton,Piccadilly[h],Brent,4,1903-06-28
4,Amersham,Metropolitan,Chiltern,9,1892-09-01


In [7]:
underground_pivot = underground_activity.pivot(index="station", columns="year", values="annual_entry_and_exit")

In [11]:
underground_pivot.dropna().to_csv("data/annual_underground_act.csv")
# underground_pivot.to_csv("data/annual_underground_act.csv")